In [1]:
import pandas as pd

In [2]:
filename = r"C:\Users\Иннокентий\Downloads\test.tsv"

In [3]:
sep='>'

In [4]:
df = pd.read_csv(filename, sep='\t', encoding='utf-8', names=['text', 'with_sep'])

In [5]:
df

,text,with_sep
0,Koqʼ ri akʼal .,K>oqʼ ri akʼal .
1,Xatqʼajik .,X>at>qʼaj>ik .
2,chi jun tinimit jawi kʼi wi kibʼantajik ri win...,chi jun tinimit jawi kʼi wi ki>bʼantajik ri wi...
3,Xa jekʼuriʼ ri ix kamik riʼ man ix kaxlan wina...,Xa jekʼuriʼ ri ix kamik riʼ man ix kaxlan wina...
4,Xraj kaʼe le unan,X>r>aj ka>ʼe le u>nan
...,...,...
137,Kʼo pus che ri wa .,Kʼo pus ch>e ri wa .
138,rumal rech che qastzij rumal ri usipanik xixko...,r>umal rech ch>e qastzij r>umal ri u>sipanik x...
139,Natural,Natural
140,kojonik pa ri utzij wa Iximulew .,kojonik pa ri u>tzij wa Iximulew .


In [6]:
kiche_anbur = "a [a], ä [ə], e [ɛ], i [i], o [o], u [u], m [m], n [n], nh [ŋ], p [p], bʼ [ɓ], t [t], tʼ [tʼ], k [k], kʼ [kʼ], q [q], qʼ [qʼ], tz [ʦ], tzʼ [ʦ’], ch [ʧ], chʼ [ʧ’], s [s], x [ʃ], j [χ], h [h], r [r], l [l], y [j], w [w], ʼ [ʔ]"
alphabet = []
for letter in kiche_anbur.split(","):
    letter = letter.split()[0]
    alphabet.append(letter)
    alphabet.append(letter[0].upper() + letter[1:])

In [7]:
print(alphabet)

['a', 'A', 'ä', 'Ä', 'e', 'E', 'i', 'I', 'o', 'O', 'u', 'U', 'm', 'M', 'n', 'N', 'nh', 'Nh', 'p', 'P', 'bʼ', 'Bʼ', 't', 'T', 'tʼ', 'Tʼ', 'k', 'K', 'kʼ', 'Kʼ', 'q', 'Q', 'qʼ', 'Qʼ', 'tz', 'Tz', 'tzʼ', 'Tzʼ', 'ch', 'Ch', 'chʼ', 'Chʼ', 's', 'S', 'x', 'X', 'j', 'J', 'h', 'H', 'r', 'R', 'l', 'L', 'y', 'Y', 'w', 'W', 'ʼ', 'ʼ']


In [8]:
alph_to_int = dict(zip(alphabet, range(len(alphabet))))
int_to_alph = dict(zip(range(len(alphabet)), alphabet))

In [9]:
def window(text, size):
    tokens = []
    i = 0
    while i < (len(text)-size+1):
        try:
            current = ''.join(text[i:i+size])
        except:
            current = 'not_in_alphabet'
        if current in alphabet:
            tokens.append(alph_to_int[current])
            i += size
        else:
            tokens.append(text[i])
            i += 1
    return tokens

In [10]:
def tokenize(text):
    tokens = list(text + 'ггг')
    for i in range(3, 0, -1):
        tokens = window(tokens, i)
    tokens_2 = []
    for token in tokens:
        if token in int_to_alph:
            token = int_to_alph[token]
        tokens_2.append(token)
    return tokens_2

In [11]:
tokenize("Entzʼonses are kʼwa")

['E',
 'n',
 'tzʼ',
 'o',
 'n',
 's',
 'e',
 's',
 ' ',
 'a',
 'r',
 'e',
 ' ',
 'kʼ',
 'w',
 'a']

In [12]:
#b, m и s значат beginning, middle и end, а s — single
def to_labels(text):
    border_tokens = sep + ".!,? "   #тут можно расширить каким-нибудь string.punctuation например
    labels = text.copy()
    labels[0] = 'b-morph'
    for i in range(len(text)-1):
        current = labels[i]
        previous = labels[i-1]
        next_ = labels[i+1]
        if text[i] in border_tokens:
            next_ = 'b-morph'
            if previous != 'b-morph':
                previous = 'e-morph'
            else:
                previous = 's-morph'
        else:
            if current != 'b-morph':
                current = 'm-morph'
        labels[i] = current
        labels[i-1] = previous
        labels[i+1] = next_
    if labels[-1] != 'b-morph':
        labels[-1] = 'e-morph'
    else:
        labels[-1] = 's-morph'
    #удаляем разделитель #очень костыльное место, так-то надо всю функцию переделать
    labels = list(filter(lambda x: x!=sep, labels))
    return labels

In [13]:
df['tokens'] = df['text'].apply(tokenize)

In [14]:
df['tokens_with_sep'] = df['with_sep'].apply(tokenize)

In [15]:
df['labels'] = df['tokens_with_sep'].apply(to_labels)

In [16]:
df

,text,with_sep,tokens,tokens_with_sep,labels
0,Koqʼ ri akʼal .,K>oqʼ ri akʼal .,"[K, o, qʼ, , r, i, , a, kʼ, a, l, , ., ]","[K, >, o, qʼ, , r, i, , a, kʼ, a, l, , .]","[s-morph, b-morph, e-morph, , b-morph, e-morp..."
1,Xatqʼajik .,X>at>qʼaj>ik .,"[X, a, t, qʼ, a, j, i, k, , ., ]","[X, >, a, t, >, qʼ, a, j, >, i, k, , .]","[s-morph, b-morph, e-morph, b-morph, m-morph, ..."
2,chi jun tinimit jawi kʼi wi kibʼantajik ri win...,chi jun tinimit jawi kʼi wi ki>bʼantajik ri wi...,"[ch, i, , j, u, n, , t, i, n, i, m, i, t, ,...","[ch, i, , j, u, n, , t, i, n, i, m, i, t, ,...","[b-morph, e-morph, , b-morph, m-morph, e-morp..."
3,Xa jekʼuriʼ ri ix kamik riʼ man ix kaxlan wina...,Xa jekʼuriʼ ri ix kamik riʼ man ix kaxlan wina...,"[X, a, , j, e, kʼ, u, r, i, ʼ, , r, i, , i,...","[X, a, , j, e, kʼ, u, r, i, ʼ, , r, i, , i,...","[b-morph, e-morph, , b-morph, m-morph, m-morp..."
4,Xraj kaʼe le unan,X>r>aj ka>ʼe le u>nan,"[X, r, a, j, , k, a, ʼ, e, , l, e, , u, n, ...","[X, >, r, >, a, j, , k, a, >, ʼ, e, , l, e, ...","[s-morph, s-morph, b-morph, e-morph, , b-morp..."
...,...,...,...,...,...
137,Kʼo pus che ri wa .,Kʼo pus ch>e ri wa .,"[Kʼ, o, , p, u, s, , ch, e, , r, i, , w, a...","[Kʼ, o, , p, u, s, , ch, >, e, , r, i, , w...","[b-morph, e-morph, , b-morph, m-morph, e-morp..."
138,rumal rech che qastzij rumal ri usipanik xixko...,r>umal rech ch>e qastzij r>umal ri u>sipanik x...,"[r, u, m, a, l, , r, e, ch, , ch, e, , q, a...","[r, >, u, m, a, l, , r, e, ch, , ch, >, e, ...","[s-morph, b-morph, m-morph, m-morph, e-morph, ..."
139,Natural,Natural,"[N, a, t, u, r, a, l]","[N, a, t, u, r, a, l]","[b-morph, m-morph, m-morph, m-morph, m-morph, ..."
140,kojonik pa ri utzij wa Iximulew .,kojonik pa ri u>tzij wa Iximulew .,"[k, o, j, o, n, i, k, , p, a, , r, i, , u, ...","[k, o, j, o, n, i, k, , p, a, , r, i, , u, ...","[b-morph, m-morph, m-morph, m-morph, m-morph, ..."


In [17]:
import pickle

In [18]:
filename = filename.split('\\')[-1]
filename = filename.split('.')[0]

In [19]:
with open (filename + '.pickle', 'bw') as f:
    pickle.dump(df, f)